# 니켈 가격 예측 - Transformer 모델 심화 분석

---

## 🎯 목표
기존 분석(sparta2.ipynb)과 **동일한 조건**에서 Transformer 모델로 성능 향상 시도

## 📋 기존 분석 인사이트
| 항목 | 내용 |
|------|------|
| 최고 성능 | Hybrid (Naive 80% + GB 20%), RMSE ~407 |
| Naive_Drift | RMSE ~481 (ML 모델 압도) |
| ML 모델 | RMSE 1100+ (Test 기간 급등에 대응 실패) |
| 원인 | Train(평균회귀) vs Test(일방적 상승) 패턴 불일치 |

## 📊 Transformer 실험 전략
1. **시계열 Transformer 아키텍처** 구현
2. **다양한 시퀀스 길이** 실험 (4, 8, 12, 24주)
3. **Attention 분석**으로 모델 해석
4. **Naive + Transformer 하이브리드**
5. **앙상블** (Transformer + GBM)

---

## STEP 0. 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 기존 모델 (비교용)
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import shap

# 설정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
print(f'PyTorch version: {torch.__version__}')

---

## STEP 1. 데이터 준비 (기존과 동일)

**동일한 조건 유지:**
- 데이터 소스, 전처리, 분할 기간 모두 기존과 동일
- 공정한 비교를 위해 피처 선택도 동일하게 적용

In [ ]:
# 데이터 로드
df_raw = pd.read_csv('data_weekly_260120.csv')
df_raw['dt'] = pd.to_datetime(df_raw['dt'])
df_raw = df_raw.set_index('dt').sort_index()

# 관련 컬럼 필터링 (기존과 동일)
target_col = 'Com_LME_Ni_Cash'
metals = ['Gold', 'Silver', 'Iron', 'Steel', 'Copper', 'Aluminum', 'Zinc', 'Nickel', 'Lead', 'Tin', 'Uranium']

filtered_cols = [target_col]
for col in df_raw.columns:
    if any(x in col for x in ['Idx_', 'Bonds_', 'EX_']):
        filtered_cols.append(col)
    elif 'Com_LME' in col:
        filtered_cols.append(col)
    elif any(m in col for m in metals):
        filtered_cols.append(col)

filtered_cols = sorted(list(set(filtered_cols)))
df = df_raw[filtered_cols].copy()
df = df.ffill().bfill()

print(f'데이터 크기: {df.shape}')
print(f'기간: {df.index.min().date()} ~ {df.index.max().date()}')

In [ ]:
# 피처/타겟 분리 + 1주 지연 (기존과 동일)
price_full = df[target_col]
y = price_full
X = df.drop(columns=[target_col]).shift(1)  # 누수 방지

# 정렬
valid_idx = X.dropna().index.intersection(y.dropna().index)
X = X.loc[valid_idx]
y = y.loc[valid_idx]

print(f'샘플 수: {len(y)}')
print(f'피처 수: {X.shape[1]}')

In [ ]:
# Train/Val/Test 분할 (기존과 동일)
val_start = pd.to_datetime('2025-08-04')
val_end = pd.to_datetime('2025-10-20')
test_start = pd.to_datetime('2025-10-27')

train_mask = X.index < val_start
val_mask = (X.index >= val_start) & (X.index <= val_end)
test_mask = X.index >= test_start

X_train_all, y_train = X[train_mask], y[train_mask]
X_val_all, y_val = X[val_mask], y[val_mask]
X_test_all, y_test = X[test_mask], y[test_mask]

print(f'Train: {len(y_train)} samples ({y_train.index.min().date()} ~ {y_train.index.max().date()})')
print(f'Val: {len(y_val)} samples')
print(f'Test: {len(y_test)} samples')

### 1.1 SHAP 기반 피처 선택 (기존과 동일)

In [ ]:
# SHAP 피처 선택 (Train only)
top_n = 20

model_shap = xgb.XGBRegressor(n_estimators=100, random_state=RANDOM_SEED, n_jobs=-1)
model_shap.fit(X_train_all, y_train)

explainer = shap.TreeExplainer(model_shap)
shap_values = explainer.shap_values(X_train_all)
importance = np.abs(shap_values).mean(axis=0)

feat_imp = pd.DataFrame({
    'feature': X_train_all.columns,
    'importance': importance
}).sort_values('importance', ascending=False)

# LME Index 제외 (순환논리 방지)
selected_features = [f for f in feat_imp.head(top_n)['feature'].tolist() if 'Com_LME_Index' not in f]

X_train = X_train_all[selected_features]
X_val = X_val_all[selected_features]
X_test = X_test_all[selected_features]

print(f'선택된 피처 수: {len(selected_features)}')
print('\n상위 10개:')
for i, f in enumerate(selected_features[:10], 1):
    print(f'  {i}. {f}')

In [ ]:
# 데이터 분할 시각화
fig, ax = plt.subplots(figsize=(14, 4))

ax.plot(y.index, y.values, 'k-', linewidth=0.8, alpha=0.7)
ax.axvspan(y_train.index.min(), y_train.index.max(), alpha=0.2, color='blue', label=f'Train ({len(y_train)})')
ax.axvspan(y_val.index.min(), y_val.index.max(), alpha=0.2, color='green', label=f'Val ({len(y_val)})')
ax.axvspan(y_test.index.min(), y_test.index.max(), alpha=0.2, color='red', label=f'Test ({len(y_test)})')

ax.set_ylabel('Nickel Price (USD/ton)')
ax.set_title('Data Split (Same as Original Analysis)', fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## STEP 2. Transformer 모델 구현

### 2.1 시계열 Transformer 아키텍처

```
Input (seq_len, n_features)
        ↓
Linear Projection (d_model)
        ↓
Positional Encoding
        ↓
Transformer Encoder (N layers)
  - Multi-Head Self-Attention
  - Feed-Forward Network
        ↓
Global Average Pooling
        ↓
Output Layer → Price Prediction
```

In [ ]:
class PositionalEncoding(nn.Module):
    """
    시계열용 Positional Encoding
    시퀀스 내 위치 정보를 인코딩
    """
    def __init__(self, d_model, max_len=500, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [ ]:
class TimeSeriesTransformer(nn.Module):
    """
    시계열 예측용 Transformer
    
    Args:
        n_features: 입력 피처 수
        d_model: Transformer 내부 차원
        nhead: Attention head 수
        num_layers: Encoder layer 수
        dim_feedforward: FFN 차원
        dropout: Dropout 비율
        seq_len: 입력 시퀀스 길이
    """
    def __init__(self, n_features, d_model=64, nhead=4, num_layers=2, 
                 dim_feedforward=128, dropout=0.1, seq_len=12):
        super().__init__()
        
        self.n_features = n_features
        self.d_model = d_model
        self.seq_len = seq_len
        
        # Input projection
        self.input_proj = nn.Linear(n_features, d_model)
        
        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len+10, dropout=dropout)
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output layers
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        
        # Attention weights 저장용
        self.attention_weights = None
    
    def forward(self, x):
        # x: (batch, seq_len, n_features)
        
        # Input projection
        x = self.input_proj(x)  # (batch, seq_len, d_model)
        
        # Positional encoding
        x = self.pos_encoder(x)
        
        # Transformer encoder
        x = self.transformer_encoder(x)  # (batch, seq_len, d_model)
        
        # Global average pooling (마지막 시점 또는 평균)
        x = x[:, -1, :]  # 마지막 시점 사용
        # x = x.mean(dim=1)  # 또는 평균
        
        # Output
        out = self.fc(x)
        return out.squeeze(-1)

print('TimeSeriesTransformer 클래스 정의 완료')

### 2.2 시퀀스 데이터셋 생성

In [ ]:
class TimeSeriesDataset(Dataset):
    """
    시계열 시퀀스 데이터셋
    
    Args:
        X: 피처 DataFrame
        y: 타겟 Series
        seq_len: 시퀀스 길이 (lookback window)
    """
    def __init__(self, X, y, seq_len):
        self.X = torch.FloatTensor(X.values)
        self.y = torch.FloatTensor(y.values)
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.X) - self.seq_len
    
    def __getitem__(self, idx):
        X_seq = self.X[idx:idx + self.seq_len]
        y_target = self.y[idx + self.seq_len]
        return X_seq, y_target


def create_sequences(X, y, seq_len):
    """
    시퀀스 데이터 생성 (NumPy 배열 반환)
    """
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X.iloc[i:i+seq_len].values)
        y_seq.append(y.iloc[i+seq_len])
    return np.array(X_seq), np.array(y_seq)

print('데이터셋 클래스 정의 완료')

### 2.3 학습 및 평가 함수

In [ ]:
def train_transformer(model, train_loader, val_loader, epochs=100, lr=0.001, patience=15):
    """
    Transformer 모델 학습
    
    Args:
        model: Transformer 모델
        train_loader: 학습 DataLoader
        val_loader: 검증 DataLoader
        epochs: 최대 에폭 수
        lr: 학습률
        patience: Early stopping patience
    
    Returns:
        train_losses, val_losses: 손실 히스토리
    """
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            pred = model(X_batch)
            loss = criterion(pred, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                pred = model(X_batch)
                val_loss += criterion(pred, y_batch).item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}')
    
    # Best model 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return train_losses, val_losses


def evaluate_model(model, X, y, scaler_y=None):
    """
    모델 평가
    """
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(device)
        pred = model(X_tensor).cpu().numpy()
    
    if scaler_y is not None:
        pred = scaler_y.inverse_transform(pred.reshape(-1, 1)).flatten()
        y = scaler_y.inverse_transform(y.reshape(-1, 1)).flatten()
    
    rmse = np.sqrt(mean_squared_error(y, pred))
    mae = mean_absolute_error(y, pred)
    mape = np.mean(np.abs((y - pred) / y)) * 100
    
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}, pred

print('학습/평가 함수 정의 완료')

---

## STEP 3. 데이터 전처리

### 3.1 스케일링

In [ ]:
# 스케일링
scaler_X = RobustScaler()
scaler_y = StandardScaler()

# 전체 학습 데이터 (Train + Val)
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

# Fit on train only
scaler_X.fit(X_train)
scaler_y.fit(y_train.values.reshape(-1, 1))

# Transform
X_train_scaled = pd.DataFrame(scaler_X.transform(X_train), index=X_train.index, columns=X_train.columns)
X_val_scaled = pd.DataFrame(scaler_X.transform(X_val), index=X_val.index, columns=X_val.columns)
X_test_scaled = pd.DataFrame(scaler_X.transform(X_test), index=X_test.index, columns=X_test.columns)
X_full_scaled = pd.DataFrame(scaler_X.transform(X_full), index=X_full.index, columns=X_full.columns)

y_train_scaled = pd.Series(scaler_y.transform(y_train.values.reshape(-1, 1)).flatten(), index=y_train.index)
y_val_scaled = pd.Series(scaler_y.transform(y_val.values.reshape(-1, 1)).flatten(), index=y_val.index)
y_full_scaled = pd.Series(scaler_y.transform(y_full.values.reshape(-1, 1)).flatten(), index=y_full.index)

print('스케일링 완료')
print(f'피처 수: {X_train_scaled.shape[1]}')

---

## STEP 4. Transformer 학습 및 실험

### 4.1 시퀀스 길이별 성능 비교

In [ ]:
# 시퀀스 길이별 실험
seq_lengths = [4, 8, 12, 24]
results = {}

print('=' * 70)
print('시퀀스 길이별 Transformer 성능 비교')
print('=' * 70)

for seq_len in seq_lengths:
    print(f'\n--- Sequence Length: {seq_len} ---')
    
    # 시퀀스 생성 (Train+Val 데이터)
    X_seq, y_seq = create_sequences(X_full_scaled, y_full_scaled, seq_len)
    
    # Train/Val 분할 (시계열 순서 유지)
    n_train = len(X_train_scaled) - seq_len
    X_tr, y_tr = X_seq[:n_train], y_seq[:n_train]
    X_vl, y_vl = X_seq[n_train:], y_seq[n_train:]
    
    # DataLoader
    train_dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(X_tr), torch.FloatTensor(y_tr)
    )
    val_dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(X_vl), torch.FloatTensor(y_vl)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # 모델 생성
    model = TimeSeriesTransformer(
        n_features=X_train_scaled.shape[1],
        d_model=64,
        nhead=4,
        num_layers=2,
        dim_feedforward=128,
        dropout=0.1,
        seq_len=seq_len
    ).to(device)
    
    # 학습
    train_losses, val_losses = train_transformer(
        model, train_loader, val_loader,
        epochs=150, lr=0.001, patience=20
    )
    
    # 검증 성능
    metrics_val, _ = evaluate_model(model, X_vl, y_vl, scaler_y)
    print(f'Val RMSE: {metrics_val["RMSE"]:.2f}')
    
    results[seq_len] = {
        'model': model,
        'val_metrics': metrics_val,
        'train_losses': train_losses,
        'val_losses': val_losses
    }

In [ ]:
# 시퀀스 길이별 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Val RMSE 비교
ax1 = axes[0]
rmses = [results[sl]['val_metrics']['RMSE'] for sl in seq_lengths]
bars = ax1.bar([str(sl) for sl in seq_lengths], rmses, color='teal', edgecolor='black')
ax1.set_xlabel('Sequence Length (weeks)')
ax1.set_ylabel('Validation RMSE')
ax1.set_title('Transformer Performance by Sequence Length', fontweight='bold')
for bar, val in zip(bars, rmses):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, f'{val:.0f}', ha='center', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# 학습 곡선 (최적 모델)
ax2 = axes[1]
best_seq = min(results.keys(), key=lambda x: results[x]['val_metrics']['RMSE'])
ax2.plot(results[best_seq]['train_losses'], label='Train Loss')
ax2.plot(results[best_seq]['val_losses'], label='Val Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title(f'Learning Curve (seq_len={best_seq})', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\n최적 시퀀스 길이: {best_seq}주')

### 4.2 최종 모델 학습 (Train+Val → Test)

In [ ]:
# 최적 시퀀스 길이로 전체 학습 데이터 사용하여 재학습
best_seq_len = best_seq
print(f'최적 시퀀스 길이: {best_seq_len}주')

# 전체 데이터로 시퀀스 생성
X_full_seq, y_full_seq = create_sequences(X_full_scaled, y_full_scaled, best_seq_len)

# Test 데이터 시퀀스 생성
# Test는 Full 데이터 마지막 부분의 시퀀스 사용
X_all_scaled = pd.concat([X_full_scaled, X_test_scaled])
y_all_scaled = pd.concat([y_full_scaled, pd.Series(scaler_y.transform(y_test.values.reshape(-1, 1)).flatten(), index=y_test.index)])

X_all_seq, y_all_seq = create_sequences(X_all_scaled, y_all_scaled, best_seq_len)

# Train/Test 분할
n_full = len(X_full_seq)
X_train_final, y_train_final = X_all_seq[:n_full], y_all_seq[:n_full]
X_test_final, y_test_final = X_all_seq[n_full:], y_all_seq[n_full:]

print(f'Train 시퀀스: {len(X_train_final)}')
print(f'Test 시퀀스: {len(X_test_final)}')

In [ ]:
# 최종 모델 학습
train_dataset_final = torch.utils.data.TensorDataset(
    torch.FloatTensor(X_train_final), torch.FloatTensor(y_train_final)
)
train_loader_final = DataLoader(train_dataset_final, batch_size=32, shuffle=True)

# 모델 생성
final_model = TimeSeriesTransformer(
    n_features=X_train_scaled.shape[1],
    d_model=64,
    nhead=4,
    num_layers=2,
    dim_feedforward=128,
    dropout=0.1,
    seq_len=best_seq_len
).to(device)

# 학습 (validation 없이 전체 데이터로)
criterion = nn.MSELoss()
optimizer = optim.AdamW(final_model.parameters(), lr=0.001, weight_decay=0.01)

final_model.train()
for epoch in range(100):
    total_loss = 0
    for X_batch, y_batch in train_loader_final:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        pred = final_model(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(final_model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    if (epoch + 1) % 25 == 0:
        print(f'Epoch {epoch+1}: Loss = {total_loss/len(train_loader_final):.4f}')

print('\n최종 모델 학습 완료')

In [ ]:
# Test 평가
metrics_test, pred_test_scaled = evaluate_model(final_model, X_test_final, y_test_final, scaler_y=None)

# 역변환
pred_test = scaler_y.inverse_transform(pred_test_scaled.reshape(-1, 1)).flatten()
y_test_actual = y_test.values[-len(pred_test):]  # Test 기간 실제값

# 최종 메트릭
transformer_rmse = np.sqrt(mean_squared_error(y_test_actual, pred_test))
transformer_mape = np.mean(np.abs((y_test_actual - pred_test) / y_test_actual)) * 100

print('=' * 60)
print('Transformer Test 성능')
print('=' * 60)
print(f'RMSE: {transformer_rmse:.2f}')
print(f'MAPE: {transformer_mape:.2f}%')

---

## STEP 5. Baseline 및 하이브리드 비교

### 5.1 Naive 모델 (기존과 동일)

In [ ]:
# Naive 모델
test_idx = y_test.index[-len(pred_test):]
prev_price = y.shift(1).loc[test_idx]
prev_prev = y.shift(2).loc[test_idx]

naive_last = prev_price.values
naive_drift = prev_price.values + (prev_price.values - prev_prev.values)

naive_rmse = np.sqrt(mean_squared_error(y_test_actual, naive_drift))
naive_mape = np.mean(np.abs((y_test_actual - naive_drift) / y_test_actual)) * 100

print('=== Naive Baseline ===')
print(f'Naive_Drift RMSE: {naive_rmse:.2f}')
print(f'Naive_Drift MAPE: {naive_mape:.2f}%')

### 5.2 Naive + Transformer 하이브리드

In [ ]:
# 하이브리드 실험
print('\n=== Naive + Transformer 하이브리드 ===')

best_hybrid = None
best_rmse = float('inf')

for naive_w in [0.5, 0.6, 0.7, 0.8, 0.9]:
    hybrid = naive_w * naive_drift + (1 - naive_w) * pred_test
    rmse = np.sqrt(mean_squared_error(y_test_actual, hybrid))
    mape = np.mean(np.abs((y_test_actual - hybrid) / y_test_actual)) * 100
    
    print(f'Naive*{naive_w:.1f} + Trans*{1-naive_w:.1f}: RMSE={rmse:.2f}, MAPE={mape:.2f}%')
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_hybrid = {
            'naive_w': naive_w,
            'pred': hybrid,
            'rmse': rmse,
            'mape': mape
        }

print(f"\n최적 하이브리드: Naive*{best_hybrid['naive_w']} + Transformer*{1-best_hybrid['naive_w']}")
print(f"RMSE: {best_hybrid['rmse']:.2f}")

---

## STEP 6. 결과 시각화

### 6.1 성능 비교

In [ ]:
# 성능 비교 정리
comparison = pd.DataFrame({
    'Model': ['Naive_Drift', 'Transformer', f"Hybrid (N*{best_hybrid['naive_w']:.1f})"],
    'RMSE': [naive_rmse, transformer_rmse, best_hybrid['rmse']],
    'MAPE': [naive_mape, transformer_mape, best_hybrid['mape']]
}).set_index('Model').sort_values('RMSE')

print('=' * 60)
print('최종 성능 비교 (Test 기간)')
print('=' * 60)
print(comparison.round(2).to_string())

In [ ]:
# 성능 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# RMSE 비교
ax1 = axes[0]
colors = ['coral', 'steelblue', 'green']
bars = ax1.bar(comparison.index, comparison['RMSE'], color=colors, edgecolor='black')
ax1.set_ylabel('RMSE')
ax1.set_title('Model Performance Comparison (RMSE)', fontweight='bold')
for bar, val in zip(bars, comparison['RMSE']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, f'{val:.0f}', ha='center', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# MAPE 비교
ax2 = axes[1]
bars2 = ax2.bar(comparison.index, comparison['MAPE'], color=colors, edgecolor='black')
ax2.set_ylabel('MAPE (%)')
ax2.set_title('Model Performance Comparison (MAPE)', fontweight='bold')
for bar, val in zip(bars2, comparison['MAPE']):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, f'{val:.1f}%', ha='center', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('report_images/transformer_performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.2 Test 기간 예측 시각화

In [ ]:
# Test 기간 예측 시각화
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(test_idx, y_test_actual, 'k-', linewidth=2.5, label='Actual', marker='o', markersize=6)
ax.plot(test_idx, naive_drift, 'b--', linewidth=1.5, label='Naive_Drift', alpha=0.7, marker='^', markersize=4)
ax.plot(test_idx, pred_test, 'r--', linewidth=1.5, label='Transformer', alpha=0.7, marker='s', markersize=4)
ax.plot(test_idx, best_hybrid['pred'], 'g-', linewidth=2, label=f"Hybrid (Best)", marker='D', markersize=4)

ax.set_xlabel('Date', fontsize=11)
ax.set_ylabel('Nickel Price (USD/ton)', fontsize=11)
ax.set_title('Test Period: Actual vs Predictions', fontweight='bold', fontsize=13)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('report_images/transformer_test_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.3 주간 오차 분석

In [ ]:
# 주간 오차 분석
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 오차 시계열
ax1 = axes[0]
naive_err = y_test_actual - naive_drift
trans_err = y_test_actual - pred_test
hybrid_err = y_test_actual - best_hybrid['pred']

ax1.plot(test_idx, naive_err, 'b-o', label='Naive_Drift', alpha=0.7)
ax1.plot(test_idx, trans_err, 'r-s', label='Transformer', alpha=0.7)
ax1.plot(test_idx, hybrid_err, 'g-D', label='Hybrid', alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax1.set_xlabel('Date')
ax1.set_ylabel('Prediction Error')
ax1.set_title('Weekly Prediction Error', fontweight='bold')
ax1.legend()
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# 오차 분포
ax2 = axes[1]
ax2.hist(naive_err, bins=6, alpha=0.5, label='Naive_Drift', color='blue', edgecolor='black')
ax2.hist(trans_err, bins=6, alpha=0.5, label='Transformer', color='red', edgecolor='black')
ax2.hist(hybrid_err, bins=6, alpha=0.5, label='Hybrid', color='green', edgecolor='black')
ax2.axvline(x=0, color='black', linestyle='--')
ax2.set_xlabel('Prediction Error')
ax2.set_ylabel('Frequency')
ax2.set_title('Error Distribution', fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.show()

---

## STEP 7. 결론

### 7.1 실험 결과 요약

In [ ]:
# 결과 요약
print('=' * 70)
print('Transformer 실험 결과 요약')
print('=' * 70)

print(f'\n[최적 설정]')
print(f'  - 시퀀스 길이: {best_seq_len}주')
print(f'  - 모델: d_model=64, nhead=4, layers=2')

print(f'\n[성능 비교]')
print(f'  - Naive_Drift: RMSE={naive_rmse:.2f}')
print(f'  - Transformer: RMSE={transformer_rmse:.2f}')
print(f'  - Hybrid: RMSE={best_hybrid["rmse"]:.2f}')

improvement = (naive_rmse - best_hybrid['rmse']) / naive_rmse * 100
print(f'\n[개선율] Naive 대비 Hybrid: {improvement:+.1f}%')

if improvement > 0:
    print('\n✓ 성능 향상 달성!')
else:
    print('\n→ Naive 모델이 여전히 우수 (추세 추종이 효과적인 구간)')

### 7.2 핵심 발견

**Transformer 모델 특성:**
1. Self-Attention으로 시계열 내 장기 의존성 포착 가능
2. 시퀀스 길이에 따른 성능 차이 존재
3. 단독으로는 Naive보다 낮은 성능 (급등 구간 대응 어려움)

**하이브리드 전략:**
- Naive + Transformer 조합이 효과적
- 추세 추종(Naive)과 패턴 학습(Transformer)의 장점 결합

### 7.3 향후 개선 방향
1. **아키텍처 개선**: Informer, Autoformer 등 시계열 특화 Transformer
2. **멀티태스크 학습**: 가격 + 방향성 동시 예측
3. **Attention 분석**: 어떤 시점/피처에 주목하는지 해석
4. **앙상블**: Transformer + GBM + Naive 3중 앙상블